# Import

In [2]:
import os
import glob
import gzip
import numpy as np
import pandas as pd

# Config variable

In [3]:
read_size = 100
sub_rate = 0
ins_rate = 0
del_rate = 0
sub_readrate = 0
ins_readrate = 0
del_readrate = 0
coverage = 1

Frequency = [4,3,2,1,1,1,1,1,1,1]

# Select genome

In [ ]:
def clear(dna):
    out = list()
    for i in range(0,len(dna)):
        if(dna[i] in ['A','C','G','T']):            
            out.append(dna[i])
    return ''.join(out)

In [ ]:
np.random.seed(0) # set random seed

files = [x[0] for x in os.walk('./out/refseq/bacteria/')][1:]
files = np.random.choice(files,len(Frequency)) # select 10 genome randomly
genoms = list()
df = pd.read_csv('./assembly_summary.txt', sep='\t',  lineterminator='\n') # for retrieve taxaid

for file in files:       
    query = file.split('/')[-1]
    taxid = int(df.loc[df['gbrs_paired_asm'] == query]['taxid']) 
    fna = glob.glob(file+"/*.fna.gz")[0]
    f = gzip.open(fna, 'rb')    # unzip fna.gz files
    file_content = str(f.read())
    f.close()     
    parts = file_content.split('>N')                      
    dna =""
    for p in parts:
        if(len(p)>len(dna)):
            dna = p    
    dna = dna.split('\\n')          
    dna = dna[1:]    
    dna = ''.join(dna)        
    dna = clear(dna)
    genoms.append([dna,taxid])    
del(df)

## Mutation in genome g*

In [24]:
for i in range(0,len(Frequency)):        
    for j in range(0,Frequency[i]):                   
        genoms.append([genoms[i][0],genoms[i][1]])        

In [25]:
def mutation(dna, rate_sub, rate_ins, rate_del):
    out = list()
    for d in dna:        
        if(np.random.rand() <= rate_sub ):
            if(d=='A'):
                l = ['C','G','T']
            elif(d=='C'):
                l = ['A','G','T']
            elif(d=='G'):
                l = ['A','C','T']
            elif(d=='T'):
                l = ['A','C','G']    
            else:
                l = ['A','C','G','T']
            out.append(np.random.choice(l))
        elif(np.random.rand() <= rate_ins ):
            out.append(d)
            out.append(np.random.choice(['A','C','G','T']))            
        elif(np.random.rand() <= rate_del ):  
            pass
        else:
            out.append(d)
    return ''.join(out)

In [26]:
for i in range(0,len(genoms)):      
    genoms[i][0] = mutation(genoms[i][0],sub_rate,ins_rate ,del_rate)

# Read simulate 
## Sampling with replacement and adding sequence error

In [5]:
file = open('metagenomic.txt','a')
c = 1  
for gen in genoms:
    print(str((c/len(genoms))*100) + "%")
    c += 1
    dna =  gen[0]
    for i in range(0, len(dna)*coverage //read_size):
        r = np.random.randint(0,len(dna)-read_size+1)
        read = dna[r:r+read_size]
       # read = mutation(read,sub_readrate ,ins_readrate ,del_readrate)        
        file.write(read + ',' + str(gen[1]) + '\n')
file.close()

10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 %
